In [1]:
from qdrant_client import QdrantClient, models
from app.core.config import settings as config
from app.utils.utils import split_docs
from app.web_loader.bs_loader import load_web_docs
from app.web_loader.bs_utils import urls
from app.db.vector_store import VectorStore
import os
import pickle
from tenacity import (
    retry,
    wait_exponential,
    stop_after_attempt,
    retry_if_exception_type,
)
import time
import requests
import numpy as np

In [2]:
client = QdrantClient(config.qdrant_url, api_key=config.qdrant_api_key)

In [3]:
client.create_collection(
    collection_name="sutd",
    vectors_config=models.VectorParams(
        size=config.embeddings_dim, distance=models.Distance.COSINE
    ),
)

True

In [4]:
# client.delete_collection(collection_name="sutd")

In [5]:
client.get_collections()

CollectionsResponse(collections=[CollectionDescription(name='sutd')])

In [6]:
CACHE_FILE = "all_docs.pkl"

if os.path.exists(CACHE_FILE):
    print("Loading cached documents...")
    with open(CACHE_FILE, "rb") as f:
        all_docs = pickle.load(f)
else:
    all_docs = load_web_docs(urls)
    print("Saving documents to cache...")
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(all_docs, f)

Loading cached documents...


In [7]:
chunks = split_docs(all_docs)

In [8]:
print(len(chunks))
print(chunks[4])

1010
Document(text='ousing Sports and recreation centre F&B and services Library About About OVERVIEW Design AI At a glance Diversity and inclusion Sustainability PEOPLE Leadership Board of Trustees President Emeritus Professorships Faculty MORE Happenings Careers at SUTD Partnering with SUTD On this page Popular searches quick links Discover SUTD The world’s first Design AI university Innovation @ SUTD AirXeed radiosonde AI @ SUTD Mustango – DALL-E for music Discover SUTD The world’s first Design AI university Pla', metadata={'source': 'https://www.sutd.edu.sg/'})


In [9]:
vector_store = VectorStore(config)
embeddings = []

In [10]:
@retry(
    retry=retry_if_exception_type((requests.exceptions.RequestException, Exception)),
    wait=wait_exponential(multiplier=1, min=4, max=60),  # Exponential backoff
    stop=stop_after_attempt(5),  # Max 5 attempts
)
def get_embedding_with_retry(text):
    time.sleep(60 / 1500)
    return vector_store.get_embeddings(text)


for i, chunk in enumerate(chunks):
    try:
        embedding = get_embedding_with_retry(chunk.text)
        embeddings.append(embedding)

        if i % 100 == 0:
            print(f"Processed {i}/{len(chunks)} chunks")

    except Exception as e:
        print(f"Failed after retries on chunk {i}: {str(e)}")
        embeddings.append(None)

print(embeddings[0])

Processed 0/1010 chunks
Processed 100/1010 chunks
Processed 200/1010 chunks
Processed 300/1010 chunks
Processed 400/1010 chunks
Processed 500/1010 chunks
Processed 600/1010 chunks
Processed 700/1010 chunks
Processed 800/1010 chunks
Processed 900/1010 chunks
Processed 1000/1010 chunks
[ 3.66798714e-02  5.31939194e-02 -7.98823982e-02 -4.60960856e-03
  4.39698771e-02 -7.52986828e-03  3.33775505e-02 -2.97323614e-02
  4.85165454e-02  1.41170500e-02 -3.43748778e-02  3.50924693e-02
  5.48899956e-02 -1.52172865e-02 -1.72513034e-02  1.69632714e-02
  4.37707342e-02  3.86628993e-02 -1.19741268e-01 -3.74139287e-02
 -4.52079065e-02 -4.29069949e-03 -9.43819340e-03 -2.92334910e-02
  4.90868045e-03 -4.82631624e-02  3.07752546e-02  2.42127776e-02
 -1.55400648e-03  1.39718372e-02  1.83774922e-02  1.83284562e-02
  4.17063199e-03  1.26396155e-03 -2.41439063e-02 -6.15790999e-03
  3.58813740e-02 -3.25388983e-02  3.38935889e-02 -2.47451738e-02
 -6.85992464e-02 -2.52290182e-02  6.49226550e-03  2.71702372e-02


In [11]:
EMBEDDINGS_CACHE_FILE = "embeddings.pkl"

with open(EMBEDDINGS_CACHE_FILE, "wb") as f:
    pickle.dump(embeddings, f)

In [12]:
EMBEDDINGS_CACHE_FILE = "embeddings.pkl"

if os.path.exists(EMBEDDINGS_CACHE_FILE):
    print("Loading cached embeddings...")
    with open(EMBEDDINGS_CACHE_FILE, "rb") as f:
        embeddings = pickle.load(f)

Loading cached embeddings...


In [13]:
document_ids = vector_store.add_documents(docs=chunks, embeddings=embeddings)

Upserting points 0 to 1000...
Upserting points 1000 to 1010...


In [14]:
collection_info = vector_store.client.get_collection(vector_store.collection_name)
print(f"Collection info: {collection_info}")

# Count the number of points in the collection
point_count = vector_store.client.count(vector_store.collection_name)
print(f"Number of documents in vector store: {point_count}")

Collection info: status=<CollectionStatus.GREEN: 'green'> optimizer_status=<OptimizersStatusOneOf.OK: 'ok'> vectors_count=None indexed_vectors_count=0 points_count=1010 segments_count=2 config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quanti

In [15]:
query = "what is sutd?"
query_embedding = vector_store.get_embeddings(query)
results = vector_store.client.query_points(
    collection_name=vector_store.collection_name,
    query=query_embedding,
    limit=5,
)
print(results)

points=[ScoredPoint(id=327, version=0, score=0.67123914, payload={'text': 't SUTD, and discover the best fit for your interests. See more Useful information Careers at SUTD Connect with us Take a virtual campus tour 8 Somapah Rd, Singapore 487372 +65 6303 6600 +65 6303 6600 Email HASS Directory Social Media Facebook X Youtube Linkedin TikTok Instagram Contact SUTD Whistleblowing Policy Terms & Conditions Privacy Statement Copyright © 2011 – 2025 We use cookies to ensure you get the best experience on our website. By selecting “I agree”, you consent to our use of cookie', 'metadata': {'source': 'https://www.sutd.edu.sg/hass/education/undergraduate/minors/digital-humanities-minor/'}}, vector=None, shard_key=None, order_value=None), ScoredPoint(id=558, version=0, score=0.6282536, payload={'text': 't SUTD Connect with us Take a virtual campus tour 8 Somapah Rd, Singapore 487372 +65 6303 6600 +65 6303 6600 Email us Directory Social Media Facebook X YouTube LinkedIn TikTok Instagram Contact 